# UnSupervisedIOHMM

In [1]:
from IOHMM import IOHMM_model
import numpy as np, pandas as pd
import torch

## Load data

In [2]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()

input_data = data[['Open', 'High', 'Low', 'Close']][:-1]


# close price of the next day
output_data = data['Close'][1:]
# to start from 0 index
output_data.index = range(len(output_data))

output_data.name = 'Close-next-day'

data = pd.concat([input_data, output_data], axis=1)

data.head()
print(len(data))


input = torch.tensor(np.array(data[['Open', 'High', 'Low', 'Close']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close-next-day']), dtype=torch.float32)

print(output[0:5])
print(input[0:5])
torch.cat((torch.tensor([1.0]), output))



3259
tensor([1.5887, 1.4640, 1.2800, 1.0740, 1.0533])
tensor([[1.2667, 1.6667, 1.1693, 1.5927],
        [1.7193, 2.0280, 1.5533, 1.5887],
        [1.6667, 1.7280, 1.3513, 1.4640],
        [1.5333, 1.5400, 1.2473, 1.2800],
        [1.3333, 1.3333, 1.0553, 1.0740]])


tensor([  1.0000,   1.5887,   1.4640,  ..., 224.5700, 234.8600, 244.4000])

# Example 1

## Set up a simple model manully

In [3]:
IOHMM = IOHMM_model(num_states=2, inputs=input[:10], outputs=output[:10], max_iter=1000, tol=1e-6)
print(IOHMM.initial_pi)
print(IOHMM.transition_matrix)
print(IOHMM.emission_matrix)
print(IOHMM.sd)

Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Parameter containing:
tensor([[[-1.3993,  0.9210,  0.3408, -2.0423, -1.3345],
         [ 1.2106, -1.2774, -0.3037, -1.8557, -1.1611]],

        [[-0.3533, -1.0925, -0.4181, -0.2205,  0.2574],
         [ 0.5611,  0.7013, -0.8395, -1.3649, -0.4215]]], requires_grad=True)
Parameter containing:
tensor([[ 0.1763, -1.6805, -1.7900, -0.3010,  0.4219],
        [-1.3935, -0.8434,  0.3669, -1.3341, -0.0123]], requires_grad=True)
Parameter containing:
tensor([1., 1.], requires_grad=True)


In [4]:
print(IOHMM._forward())
torch.sum(IOHMM._forward(), dim=1)

tensor([[1.2872e-03, 9.9871e-01],
        [3.4013e-06, 1.0000e+00],
        [1.9425e-04, 9.9981e-01],
        [2.1546e-03, 9.9785e-01],
        [1.7199e-02, 9.8280e-01],
        [2.4432e-01, 7.5568e-01],
        [1.9091e-01, 8.0909e-01],
        [1.4835e-01, 8.5165e-01],
        [1.3585e-01, 8.6415e-01],
        [1.2060e-01, 8.7940e-01]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [5]:
print(IOHMM._backward())
torch.sum(IOHMM._backward(), dim=1)

tensor([[3.4013e-06, 1.0000e+00],
        [1.9425e-04, 9.9981e-01],
        [2.1546e-03, 9.9785e-01],
        [1.7199e-02, 9.8280e-01],
        [2.4432e-01, 7.5568e-01],
        [1.9091e-01, 8.0909e-01],
        [1.4835e-01, 8.5165e-01],
        [1.3585e-01, 8.6415e-01],
        [1.2060e-01, 8.7940e-01],
        [1.7637e-03, 9.9824e-01]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [6]:
print(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()))
torch.sum(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()), dim=1)

tensor([[4.3838e-09, 1.0000e+00],
        [6.6082e-10, 1.0000e+00],
        [4.1950e-07, 1.0000e+00],
        [3.7785e-05, 9.9996e-01],
        [5.6262e-03, 9.9437e-01],
        [7.0883e-02, 9.2912e-01],
        [3.9479e-02, 9.6052e-01],
        [2.6653e-02, 9.7335e-01],
        [2.1104e-02, 9.7890e-01],
        [2.4225e-04, 9.9976e-01]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [7]:
print(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()))
a = torch.sum(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()), axis=1)
torch.sum(a, axis=1)

tensor([[[3.2472e-07, 5.0000e-01],
         [3.2472e-07, 5.0000e-01]],

        [[2.3239e-07, 5.0000e-01],
         [2.3239e-07, 5.0000e-01]],

        [[5.3583e-09, 5.0000e-01],
         [5.3583e-09, 5.0000e-01]],

        [[2.1337e-06, 5.0000e-01],
         [2.1337e-06, 5.0000e-01]],

        [[3.4733e-04, 4.9965e-01],
         [3.4733e-04, 4.9965e-01]],

        [[1.6853e-03, 4.9831e-01],
         [1.6853e-03, 4.9831e-01]],

        [[2.3589e-02, 4.7641e-01],
         [2.3589e-02, 4.7641e-01]],

        [[1.6797e-02, 4.8320e-01],
         [1.6797e-02, 4.8320e-01]],

        [[1.1155e-02, 4.8884e-01],
         [1.1155e-02, 4.8884e-01]],

        [[1.3615e-04, 4.9986e-01],
         [1.3615e-04, 4.9986e-01]]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [8]:
IOHMM._baum_welch()

## See the training results

In [9]:
print("Initial pi")
print(IOHMM.initial_pi) # to normalize
print("Transition matrix")
print(IOHMM.transition_matrix)
print("Emission matrix")
print(IOHMM.emission_matrix)
print("Sd")
print(IOHMM.sd)

Initial pi
Parameter containing:
tensor([8.7242e-01, 1.0083e+07], requires_grad=True)
Transition matrix
Parameter containing:
tensor([[[-1.3993,  0.9210,  0.3408, -2.0423, -1.3345],
         [ 1.2106, -1.2774, -0.3037, -1.8557, -1.1611]],

        [[-0.3533, -1.0925, -0.4181, -0.2205,  0.2574],
         [ 0.5611,  0.7013, -0.8395, -1.3649, -0.4215]]], requires_grad=True)
Emission matrix
Parameter containing:
tensor([[ 0.1763, -1.6805, -1.7900, -0.3010,  0.4219],
        [-1.3935, -0.8434,  0.3669, -1.3341, -0.0123]], requires_grad=True)
Sd
Parameter containing:
tensor([1., 1.], requires_grad=True)
